# Preprocess data of COVID-19 for the analysis

In [71]:
import os
import sys
import numpy as np
import pandas as pd
import urllib.request
from github import Github
from getpass import getpass

from IPython.display import clear_output, display

## Login to GitHub

In [4]:
username = 'masterdesky'
password = getpass()

 ································


In [7]:
# Login
g = Github(username, password)

# Get the COVID-19 repository of CSSEGISandData
repo = g.get_repo('CSSEGISandData/COVID-19')

## Load data from the GitHub repository of CSSEGISandData

In [44]:
# The root directory of the COVID datasets in the CSSEGISandData's repository
_root = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/'

In [45]:
# Get the names of the directories, which contain the daily reports of the pandemic
# There are two different directories:
#   - One contains data for the US
#   - The other one contains data for the rest of the world
world_repo = repo.get_contents('./csse_covid_19_data/csse_covid_19_daily_reports/')
us_repo = repo.get_contents('./csse_covid_19_data/csse_covid_19_daily_reports_us/')

### Gather all URLs where the daily datasets are stored in raw format

In [46]:
daily_world_raw = []
daily_us_raw = []

for d in world_repo:
    _name = d.path
    if '.csv' in _name:
        daily_world_raw.append(_root + _name)

for d in us_repo:
    _name = d.path
    if '.csv' in _name:
        daily_us_raw.append(_root + _name)

### Check a some datatables to inspect their structure

In [65]:
url_test = daily_world_raw[5]
with urllib.request.urlopen(url_test) as url:
    data_test = pd.read_csv(url, sep=',', comment='#', header=0)

In [66]:
data_test.head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,Hubei,Mainland China,1/27/20 23:59,1423,76.0,45.0
1,Guangdong,Mainland China,1/27/20 23:59,151,NaN,4.0
2,Zhejiang,Mainland China,1/27/20 23:59,128,NaN,1.0
3,Henan,Mainland China,1/27/20 23:59,128,1.0,NaN
4,Chongqing,Mainland China,1/27/20 23:59,110,NaN,NaN


In [67]:
data_test.tail()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
46,NaN,Cambodia,1/27/20 23:59,1,NaN,NaN
47,NaN,Sri Lanka,1/27/20 23:59,1,NaN,NaN
48,NaN,Ivory Coast,1/27/20 23:59,1,NaN,NaN
49,New South Wales,Australia,1/27/20 23:59,4,NaN,NaN
50,Victoria,Australia,1/27/20 23:59,1,NaN,NaN


In [85]:
url_test = daily_world_raw[-1]
with urllib.request.urlopen(url_test) as url:
    data_test = pd.read_csv(url, sep=',', comment='#', header=0)

In [86]:
data_test.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-08-30 04:28:22,33.93911,67.709953,38143,1402,29063,7678.0,Afghanistan,97.982598,3.675642
1,NaN,NaN,NaN,Albania,2020-08-30 04:28:22,41.15330,20.168300,9279,275,5020,3984.0,Albania,322.433804,2.963681
2,NaN,NaN,NaN,Algeria,2020-08-30 04:28:22,28.03390,1.659600,43781,1491,30717,11573.0,Algeria,99.840271,3.405587
3,NaN,NaN,NaN,Andorra,2020-08-30 04:28:22,42.50630,1.521800,1124,53,902,169.0,Andorra,1454.733709,4.715302
4,NaN,NaN,NaN,Angola,2020-08-30 04:28:22,-11.20270,17.873900,2551,107,1041,1403.0,Angola,7.761757,4.194434


In [87]:
data_test.tail()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
3945,NaN,NaN,NaN,West Bank and Gaza,2020-08-30 04:28:22,31.952200,35.233200,21668,147,14631,6890.0,West Bank and Gaza,424.744816,0.678420
3946,NaN,NaN,NaN,Western Sahara,2020-08-30 04:28:22,24.215500,-12.885800,10,1,8,1.0,Western Sahara,1.674116,10.000000
3947,NaN,NaN,NaN,Yemen,2020-08-30 04:28:22,15.552727,48.516388,1946,563,1113,270.0,Yemen,6.524516,28.931141
3948,NaN,NaN,NaN,Zambia,2020-08-30 04:28:22,-13.133897,27.849332,11902,284,11092,526.0,Zambia,64.741234,2.386154
3949,NaN,NaN,NaN,Zimbabwe,2020-08-30 04:28:22,-19.015438,29.154857,6406,196,5056,1154.0,Zimbabwe,43.100528,3.059632


The structure heavily changed since the first data release, and thus we need to manually bodge it together to get a usable dataset.

### Gather all daily dataset

In [74]:
data_tables_world = []
data_tables_us = []

for raw_url in daily_world_raw:
    with urllib.request.urlopen(raw_url) as url:
        sys.stdout.flush()
        sys.stdout.write('\rCurrent file (world): {}'.format(raw_url.split('/')[-1]))
        data_tables_world.append(pd.read_csv(url, sep=',', comment='#', header=0))
for raw_url in daily_us_raw:
    with urllib.request.urlopen(raw_url) as url:
        sys.stdout.flush()
        sys.stdout.write('\rCurrent file (US): {}'.format(raw_url.split('/')[-1]))
        data_tables_us.append(pd.read_csv(url, sep=',', comment='#', header=0))
clear_output()
print('FINISHED!')

FINISHED!


## Standardize data tables

Names of countries and regions were changed numerous times in the daily reports. In order to properly aggregate the data values for each country or region, we need to create a dictionary that contains all the different occurrences of their names. After that we can then manually match the different names for the same country/region.

In [112]:
countries = set()
# This problem only affects the global data. US state names weren't changed (yet).
for df in data_tables_world:
    cols = df.columns
    for idx, c in enumerate(cols):
        # Since the column names are also changed in the past, we need
        # to find first the column, which stores country/region names
        if 'region' in c.lower():
            break
    region_col = cols[idx]
    for c in df[region_col].values:
        countries.add(c)

In [ ]:
{'',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 "",
 '',
 'Cruise Ship',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Hong Kong',
 'Hong Kong SAR',
 '',
 '',
 '',
 '',
 
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Macao SAR',
 'Macau',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'Norway',
 'Oman',
 'Others',
 'Pakistan',
 'Palestine',
 'Panama',
 'Papua New Guinea',
 'Paraguay',
 'Peru',
 'Philippines',
 'Poland',
 'Portugal',
 'Puerto Rico',
 'Qatar',
 'Republic of Ireland',
 '',
 'Republic of Moldova',
 '',
 'Reunion',
 'Romania',
 'Russia',
 'Russian Federation',
 'Rwanda',
 'Saint Barthelemy',
 'Saint Kitts and Nevis',
 'Saint Lucia',
 'Saint Martin',
 'Saint Vincent and the Grenadines',
 'San Marino',
 'Sao Tome and Principe',
 'Saudi Arabia',
 'Senegal',
 'Serbia',
 'Seychelles',
 'Sierra Leone',
 'Singapore',
 'Slovakia',
 'Slovenia',
 'Somalia',
 'South Africa',
 'South Korea',
 'South Sudan',
 'Spain',
 'Sri Lanka',
 'St. Martin',
 'Sudan',
 'Suriname',
 'Sweden',
 'Switzerland',
 'Syria',
 'Taipei and environs',
 'Taiwan',
 'Taiwan*',
 'Tajikistan',
 'Tanzania',
 'Thailand',
 'The Bahamas',
 'The Gambia',
 'Timor-Leste',
 'Togo',
 'Trinidad and Tobago',
 'Tunisia',
 'Turkey',
 'UK',
 'US',
 'Uganda',
 'Ukraine',
 'United Arab Emirates',
 'United Kingdom',
 'Uruguay',
 'Uzbekistan',
 'Vatican City',
 'Venezuela',
 'Viet Nam',
 'Vietnam',
 'West Bank and Gaza',
 'Western Sahara',
 'Yemen',
 'Zambia',
 'Zimbabwe',
 'occupied Palestinian territory'}

In [88]:
column_names = ['Province_State', 'Country_Region', 'Last_Update', 'Latitude', 'Longitude', 'Confirmed', 'Recovered', 'Deaths']

region_formats = {
    'Afghanistan': ['Afghanistan'],
    'Albania' : ['Albania'],
    'Algeria' : ['Algeria'],
    'Andorra' : ['Andorra'],
    'Angola' : ['Angola'],
    'Antigua and Barbuda' : ['Antigua and Barbuda'],
    'Argentina' : ['Argentina'],
    'Armenia' : ['Armenia'],
    'Aruba' : ['Aruba'],
    'Australia' : ['Australia'],
    'Austria' : ['Austria'],
    'Azerbaijan' : [' Azerbaijan', 'Azerbaijan'],
    'Bahamas' : ['Bahamas', 'Bahamas, The'],
    'Bahrain' : ['Bahrain'],
    'Bangladesh' : ['Bangladesh'],
    'Barbados' : ['Barbados'],
    'Belarus' : ['Belarus'],
    'Belgium' : ['Belgium'],
    'Belize' : ['Belize'],
    'Benin' : ['Benin'],
    'Bhutan' : ['Bhutan'],
    'Bolivia' : ['Bolivia'],
    'Bosnia and Herzegovina' : ['Bosnia and Herzegovina'],
    'Botswana' : ['Botswana'],
    'Brazil' : ['Brazil'],
    'Brunei' : ['Brunei'],
    'Bulgaria' : ['Bulgaria'],
    'Burkina Faso' : ['Burkina Faso'],
    'Burma' : ['Burma'],
    'Burundi' : ['Burundi'],
    'Cabo Verde' : ['Cabo Verde'],
    'Cambodia' : ['Cambodia'],
    'Cameroon' : ['Cameroon'],
    'Canada' : ['Canada'],
    'Cape Verde' : ['Cape Verde'],
    'Cayman Islands' : ['Cayman Islands'],
    'Central African Republic' : ['Central African Republic'],
    'Chad' : ['Chad'],
    'Channel Islands' : ['Channel Islands', 'Guernsey', 'Jersey'],
    'Chile' : ['Chile'],
    'China' : ['China', 'Mainland China'],
    'Colombia' : ['Colombia'],
    'Comoros' : ['Comoros'],
    'Congo (Brazzaville)' : ['Congo (Brazzaville)'],
    'Congo (Kinshasa)' : ['Congo (Kinshasa)', 'Republic of the Congo'],
    'Costa Rica' : ['Costa Rica'],
    "Cote d'Ivoire" : ["Cote d'Ivoire"],
    'Croatia' : ['Croatia'],
    'Cuba' : ['Cuba'],
    'Curacao' : ['Curacao'],
    'Cyprus' : ['Cyprus'],
    'Czechia' : ['Czech Republic', 'Czechia'],
    'Denmark' : ['Denmark'],
    'Diamond Princess' : ['Diamond Princess'],
    'Djibouti' : ['Djibouti'],
    'Dominica' : ['Dominica'],
    'Dominican Republic' : ['Dominican Republic'],
    'Ecuador' : ['Ecuador'],
    'Egypt' : ['Egypt'],
    'El Salvador' : ['El Salvador'],
    'Equatorial Guinea' : ['Equatorial Guinea'],
    'Eritrea' : ['Eritrea'],
    'Estonia' : ['Estonia'],
    'Eswatini' : ['Eswatini'],
    'Ethiopia' : ['Ethiopia'],
    'Faroe Islands' : ['Faroe Islands'],
    'Fiji' : ['Fiji'],
    'Finland' : ['Finland'],
    'France' : ['France'],
    'French Guiana': ['French Guiana'],
    'Gabon' : ['Gabon'],
    'Gambia' : ['Gambia', 'Gambia, The'],
    'Georgia' : ['Georgia'],
    'Germany' : ['Germany'],
    'Ghana' : ['Ghana'],
    'Gibraltar' : ['Gibraltar'],
    'Greece' : ['Greece'],
    'Greenland' : ['Greenland'],
    'Grenada' : ['Grenada'],
    'Guadeloupe' : ['Guadeloupe'],
    'Guam' : ['Guam'],
    'Guatemala' : ['Guatemala'],
    'Guinea' : ['Guinea'],
    'Guinea-Bissau' : ['Guinea-Bissau'],
    'Guyana' : ['Guyana'],
    'Haiti' : ['Haiti'],
    'Holy See' : ['Holy See'],
    'Honduras' : ['Honduras'],
    'Hungary' : ['Hungary'],
    'Iceland' : ['Iceland'],
    'India' : ['India'],
    'Indonesia' : ['Indonesia'],
    'Iran' : ['Iran', 'Iran (Islamic Republic of)'],
    'Iraq' : ['Iraq'],
    'Ireland' : ['Ireland'],
    'Israel' : ['Israel'],
    'Italy' : ['Italy'],
    'Ivory Coast' : ['Ivory Coast'],
    'Jamaica' : ['Jamaica'],
    'Japan' : ['Japan'],
    'Jordan' : ['Jordan'],
    'Kazakhstan' : ['Kazakhstan'],
    'Kenya' : ['Kenya'],
    'Korea, South' : ['Korea, South', 'Republic of Korea'],
    'Kosovo' : ['Kosovo'],
    'Kuwait' : ['Kuwait'],
    'Kyrgyzstan' : ['Kyrgyzstan'],
    'Laos' : ['Laos'],
    'Latvia' : ['Latvia'],
    'Lebanon' : ['Lebanon'],
    'Lesotho' : ['Lesotho'],
    'Liberia' : ['Liberia'],
    'Libya' : ['Libya'],
    'Liechtenstein' : ['Liechtenstein'],
    'Lithuania' : ['Lithuania'],
    'Luxembourg' : ['Luxembourg'],
    'MS Zaandam' : ['MS Zaandam'],
    'Madagascar' : ['Madagascar'],
    'Malawi' : ['Malawi'],
    'Malaysia' : ['Malaysia'],
    'Maldives' : ['Maldives'],
    'Mali' : ['Mali'],
    'Malta' : ['Malta'],
    'Martinique' : ['Martinique'],
    'Mauritania' : ['Mauritania'],
    'Mauritius' : ['Mauritius'],
    'Mayotte' : ['Mayotte'],
    'Mexico' : ['Mexico'],
    'Moldova' : ['Moldova'],
    'Monaco' : ['Monaco'],
    'Mongolia' : ['Mongolia'],
    'Montenegro' : ['Montenegro'],
    'Morocco' : ['Morocco'],
    'Mozambique' : ['Mozambique'],
    'Namibia' : ['Namibia'],
    'Nepal' : ['Nepal'],
    'Netherlands' : ['Netherlands'],
    'New Zealand' : ['New Zealand'],
    'Nicaragua' : ['Nicaragua'],
    'Niger' : ['Niger'],
    'Nigeria' : ['Nigeria'],
    'North Ireland' : ['North Ireland'],
    'North Macedonia' : ['North Macedonia'],
    'Norway' : [],
    'Oman' : [],
    'Pakistan' : [],
    'Panama' : [],
    'Papua New Guinea' : [],
    'Paraguay' : [],
    'Peru' : [],
    'Philippines' : [],
    'Poland' : [],
    'Portugal' : [],
    'Qatar' : [],
    'Romania' : [],
    'Russia' : [],
    'Rwanda' : [],
    'Saint Kitts and Nevis' : [],
    'Saint Lucia' : [],
    'Saint Vincent and the Grenadines' : [],
    'San Marino' : [],
    'Sao Tome and Principe' : [],
    'Saudi Arabia' : [],
    'Senegal' : [],
    'Serbia' : [],
    'Seychelles' : [],
    'Sierra Leone' : [],
    'Singapore' : [],
    'Slovakia' : [],
    'Slovenia' : [],
    'Somalia' : [],
    'South Africa' : [],
    'South Sudan' : [],
    'Spain' : [],
    'Sri Lanka' : [],
    'Sudan' : [],
    'Suriname' : [],
    'Sweden' : [],
    'Switzerland' : [],
    'Syria' : [],
    'Taiwan*' : [],
    'Tajikistan' : [],
    'Tanzania' : [],
    'Thailand' : [],
    'Timor-Leste' : ['East Timor', ],
    'Togo' : [],
    'Trinidad and Tobago' : [],
    'Tunisia' : [],
    'Turkey' : [],
    'US' : [],
    'Uganda' : [],
    'Ukraine' : [],
    'United Arab Emirates' : [],
    'United Kingdom' : [],
    'Uruguay' : [],
    'Uzbekistan' : [],
    'Venezuela' : [],
    'Vietnam' : [],
    'West Bank and Gaza' : [],
    'Western Sahara' : [],
    'Yemen' : [],
    'Zambia' : [],
    'Zimbabwe' : []
}

In [90]:
np.unique(data_test['Country/Region'])

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
       'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Grenada', 'Guatemala', 'Guin